![pretraining](https://img-blog.csdnimg.cn/676906f508114327b11c97ec0612008e.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

# 持续预训练
### DAPT（Domain-Adaptive Pretraining）：领域自适应预训练
### TAPT（Task-Adaptive Pretraining）: 任务自适应预训练

![预训练和微调](https://img-blog.csdnimg.cn/4ce443797920488c817e5eeba0023216.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [ ]:
! pip install --upgrade torch==1.6.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 748.8MB 10.7MB/s eta 0:00:01     |████▊                           | 110.0MB 11.6MB/s eta 0:00:55
  Found existing installation: torch 1.4.0
    Uninstalling torch-1.4.0:
      Successfully uninstalled torch-1.4.0


In [ ]:
! pip install torchvision==0.7.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 5.9MB 15.6MB/s eta 0:00:01
  Found existing installation: torchvision 0.5.0
    Uninstalling torchvision-0.5.0:
      Successfully uninstalled torchvision-0.5.0


In [ ]:
! pip install transformers==4.0.1

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 1.4MB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 2.9MB 14.1MB/s eta 0:00:01
     |████████████████████████████████| 40kB 16.3MB/s eta 0:00:01


In [ ]:
! pip install numpy==1.17.0

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 20.3MB 8.9MB/s eta 0:00:011
ERROR: tensorflow-gpu 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.2 which is incompatible.
ERROR: parl 1.3.2 has requirement cloudpickle==1.2.1, but you'll have cloudpickle 1.5.0 which is incompatible.
ERROR: paddlepaddle-gpu 1.8.0.post97 has requirement scipy<=1.3.1; python_version >= "3.5", but you'll have scipy 1.5.2 which is incompatible.
  Found existing installation: numpy 1.19.1
    Uninstalling numpy-1.19.1:
      Successfully uninstalled numpy-1.19.1


In [ ]:
import random
import numpy as np
import torch
from transformers import logging
def seed_everything(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

logging.set_verbosity_info()
seed_everything(2021)

2021

In [ ]:
corpus_path = 'data/data100821'
model_path = 'data/data94445'
output_dir = '.'

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(model_path)

Model name 'data/data94445' not found in model shortcut name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, TurkuNLP/bert-base-finnish-cased-v1, TurkuNLP/bert-base-finnish-uncased-v1, wietsedv/bert-base-dutch-cased). Assuming 'data/data94445' is a path, a model identifier, or url to a directory containing tokenizer files.
Didn't find file data/data94445/added_tokens.json. We won't load it.
Didn't find file data/data94445/special_tokens_map.json. We won't load it.
Didn't find file data/data94445/tokenizer_config.json. We won't load it.
Didn't find file data/data94445/tokenizer.j

In [8]:
import json
import pandas as pd
def parse_data(path, data_type='train'):
    sentence_a = []
    sentence_b = []
    labels = []
    with open(path, 'r', encoding='utf8') as f:
        for line in tqdm(f.readlines(), desc=f'Reading {data_type} data'):
            line = json.loads(line)
            sentence_a.append(line['sentence1'])
            sentence_b.append(line['sentence2'])
            if data_type != 'test':
                labels.append(int(line['label']))
            else:
                labels.append(0)
    df = pd.DataFrame(zip(sentence_a, sentence_b, labels), columns=['text_a', 'text_b', 'labels'])
    return df

In [9]:
from collections import defaultdict
from tqdm import tqdm
import os
def read_data(corpus_path, tokenizer):
    train_df = parse_data(os.path.join(corpus_path, 'train.json'), data_type='train')
    dev_df = parse_data(os.path.join(corpus_path, 'dev.json'), data_type='dev')
    test_df = parse_data(os.path.join(corpus_path, 'test.json'), data_type='test')
    
    train_df.append(dev_df)
    train_df.append(test_df)

    inputs = defaultdict(list)
    for _, row in tqdm(train_df.iterrows(), desc=f'Preprocessing train data', total=len(train_df)):
        inputs_dict = tokenizer.encode_plus(row[0] + row[1], add_special_tokens=True,
                                            return_token_type_ids=True, return_attention_mask=True)
        inputs['input_ids'].append(inputs_dict['input_ids'])
        inputs['token_type_ids'].append(inputs_dict['token_type_ids'])
        inputs['attention_mask'].append(inputs_dict['attention_mask'])

    return inputs

In [10]:
data = read_data(corpus_path, tokenizer)

Preprocessing train data: 100%|██████████| 34334/34334 [00:15<00:00, 2210.26it/s]


In [11]:
from torch.utils.data import Dataset
class AFQMCDataset(Dataset):

    def __init__(self, data_dict):
        super(AFQMCDataset, self).__init__()
        self.data_dict = data_dict

    def __getitem__(self, index):
        data = (self.data_dict['input_ids'][index], self.data_dict['token_type_ids'][index],
                self.data_dict['attention_mask'][index])
        return data

    def __len__(self):
        return len(self.data_dict['input_ids'])

In [12]:
train_dataset = AFQMCDataset(data)

```
from transformers import BertTokenizerFast, BertForMaskedLM
from transformers import LineByLineTextDataset, Trainer, TrainingArguments, DataCollatorForLanguageModeling
def train(self):
    # model_path是预训练模型的位置
    # 分词 BertTokenizerFast
    tokenizer = BertTokenizerFast.from_pretrained(model_path, max_len=512)
    # 定义一个模型 BertForMaskedLM
    model = BertForMaskedLM.from_pretrained(model_path)
    # daseset LineByLineTextDataset
    dataset = LineByLineTextDataset(
        tokenizer=tokenizer,
        file_path="语料文件.txt",
        block_size=128,
    )
    # collator DataCollatorForLanguageModeling
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer, mlm=True, mlm_probability=0.10
    )
    # TrainingArguments 训练参数，使用的接口类
    training_args = TrainingArguments(
        output_dir="xxxx",
        ...各种参数，参见文档或者源码
    )
    
    # 模型训练用Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    trainer.train()
    trainer.save_model("保存位置")
```

In [13]:
from transformers import BertForMaskedLM
model = BertForMaskedLM.from_pretrained(model_path)

loading configuration file data/data94445/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "directionality": "bidi",
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_past": true,
  "pad_token_id": 1,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "type_vocab_size": 2,
  "vocab_size": 21128
}

loading weights file data/data94445/pytorch_model.bin
Some weights of the model checkpoint at data/data94445 were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.se

In [2]:
import numpy as np
# N -Gram N: 1,2,3
ngrams = np.arange(1, 4, dtype=np.int64)
print(ngrams)
pvals = 1. / np.arange(1, 4)
print(pvals)
pvals /= pvals.sum(keepdims=True)
print(pvals)
pvals = pvals[::-1]
print(pvals)

[1 2 3]
[1.         0.5        0.33333333]
[0.54545455 0.27272727 0.18181818]
[0.18181818 0.27272727 0.54545455]


In [4]:
cand_indexes = [[1],[2],[3],[4],[5],[6],[7],[8]]
# ngrams [1,2,3]
ngram_indexes = []
for idx in range(len(cand_indexes)):
    ngram_index = []
    for n in ngrams:
        ngram_index.append(cand_indexes[idx:idx + n])
        ngram_indexes.append(ngram_index)

print(ngram_indexes)

[[[[1]], [[1], [2]], [[1], [2], [3]]], [[[1]], [[1], [2]], [[1], [2], [3]]], [[[1]], [[1], [2]], [[1], [2], [3]]], [[[2]], [[2], [3]], [[2], [3], [4]]], [[[2]], [[2], [3]], [[2], [3], [4]]], [[[2]], [[2], [3]], [[2], [3], [4]]], [[[3]], [[3], [4]], [[3], [4], [5]]], [[[3]], [[3], [4]], [[3], [4], [5]]], [[[3]], [[3], [4]], [[3], [4], [5]]], [[[4]], [[4], [5]], [[4], [5], [6]]], [[[4]], [[4], [5]], [[4], [5], [6]]], [[[4]], [[4], [5]], [[4], [5], [6]]], [[[5]], [[5], [6]], [[5], [6], [7]]], [[[5]], [[5], [6]], [[5], [6], [7]]], [[[5]], [[5], [6]], [[5], [6], [7]]], [[[6]], [[6], [7]], [[6], [7], [8]]], [[[6]], [[6], [7]], [[6], [7], [8]]], [[[6]], [[6], [7]], [[6], [7], [8]]], [[[7]], [[7], [8]], [[7], [8]]], [[[7]], [[7], [8]], [[7], [8]]], [[[7]], [[7], [8]], [[7], [8]]], [[[8]], [[8]], [[8]]], [[[8]], [[8]], [[8]]], [[[8]], [[8]], [[8]]]]


In [9]:
for cand_index_set in ngram_indexes:
    n = np.random.choice(ngrams[:len(cand_index_set)],
                                 p=pvals[:len(cand_index_set)] / pvals[:len(cand_index_set)].sum(keepdims=True))
    print(n)
    #
    print(cand_index_set[n - 1])
    index_set = sum(cand_index_set[n - 1], [])
    print(index_set)
    break

3
[[1], [2], [3]]
[1, 2, 3]


In [14]:
from typing import List, Tuple
class Collator:
    def __init__(self, max_seq_len: int, tokenizer: BertTokenizer, mlm_probability=0.15):
        self.max_seq_len = max_seq_len
        self.tokenizer = tokenizer
        self.mlm_probability = mlm_probability
        self.special_token_ids = {tokenizer.cls_token_id, tokenizer.sep_token_id}

    def pad_and_truncate(self, input_ids_list, token_type_ids_list,
                         attention_mask_list, max_seq_len):
        input_ids = torch.zeros((len(input_ids_list), max_seq_len), dtype=torch.long)
        token_type_ids = torch.zeros_like(input_ids)
        attention_mask = torch.zeros_like(input_ids)
        for i in range(len(input_ids_list)):
            seq_len = len(input_ids_list[i])
            if seq_len <= max_seq_len:
                input_ids[i, :seq_len] = torch.tensor(input_ids_list[i], dtype=torch.long)
                token_type_ids[i, :seq_len] = torch.tensor(token_type_ids_list[i], dtype=torch.long)
                attention_mask[i, :seq_len] = torch.tensor(attention_mask_list[i], dtype=torch.long)
            else:
                input_ids[i] = torch.tensor(input_ids_list[i][:max_seq_len - 1] + [self.tokenizer.sep_token_id],
                                            dtype=torch.long)
                token_type_ids[i] = torch.tensor(token_type_ids_list[i][:max_seq_len], dtype=torch.long)
                attention_mask[i] = torch.tensor(attention_mask_list[i][:max_seq_len], dtype=torch.long)
        return input_ids, token_type_ids, attention_mask

    def _ngram_mask(self, input_ids, max_seq_len, seed):
        
        cand_indexes = []
        
        # 哪些不是 special_token, 并记录不是 special_token 的位置，special_token不需要被预测
        for (i, id_) in enumerate(input_ids):
            if id_ in self.special_token_ids: # [CLS] [SEP]
                continue
            cand_indexes.append([i])
            # cand_indexes [[1],[2]]
        
        # 要 mask 的数量， BERT中被 MASK 的比例15% 
        num_to_predict = max(1, int(round(len(input_ids) * self.mlm_probability)))

        ngrams = np.arange(1, 4, dtype=np.int64)
        pvals = 1. / np.arange(1, 4)
        
        pvals /= pvals.sum(keepdims=True)
       
        pvals = pvals[::-1]
        
        # 1，2，3gram
        ngram_indexes = []
        for idx in range(len(cand_indexes)):
            ngram_index = []
            for n in ngrams:
                ngram_index.append(cand_indexes[idx:idx + n])
            ngram_indexes.append(ngram_index)
        
        np.random.shuffle(ngram_indexes)
        
        # covered_indexes mask 掉 n-gram的集合
        covered_indexes = set()
        for cand_index_set in ngram_indexes:
            # 要 mask 的数量 >= 规定的mask数量
            if len(covered_indexes) >= num_to_predict:
                break
            

            if not cand_index_set:
                continue

            # 按照概率挑选 n-gram n=1,2,3
            n = np.random.choice(ngrams[:len(cand_index_set)],
                                 p=pvals[:len(cand_index_set)] / pvals[:len(cand_index_set)].sum(keepdims=True))

            # 举例 n=3, 代表的是3-gram, 将[[1], [2], [3]] -> [1,2,3]
            index_set = sum(cand_index_set[n - 1], [])
            n -= 1

            # 控制要mask词的数量 不超过 num_to_predict
            while len(covered_indexes) + len(index_set) > num_to_predict:
                if n == 0:
                    break
                
                # 举例，去2-gram
                index_set = sum(cand_index_set[n - 1], [])
                n -= 1
            
            # 如果 经过 while 仍没找到，说明要重新找
            if len(covered_indexes) + len(index_set) > num_to_predict:
                continue

            is_any_index_covered = False

            # 如果该索引已经在covered_indexes（已经选中mask的索引），则不用再次被mask 
            for index in index_set:
                if index in covered_indexes:
                    is_any_index_covered = True
                    break

            if is_any_index_covered:
                continue
            
            # 如果该索引不在covered_indexes（已经选中mask的索引），则需要被mask 
            for index in index_set:
                covered_indexes.add(index)

        mask_labels = [1 if i in covered_indexes else 0 for i in range(len(input_ids))]
        mask_labels += [0] * (max_seq_len - len(mask_labels))

        # input_ids [1,2,3,4,5,6,7]  7 *0.15=1.05?
        # mask_labels [0,0,1,0,0,0,0]
        return torch.tensor(mask_labels[:max_seq_len])

    def ngram_mask(self, input_ids_list, max_seq_len):
        mask_labels = []
        for i, input_ids in enumerate(input_ids_list):
            mask_label = self._ngram_mask(input_ids, max_seq_len, seed=i)
            mask_labels.append(mask_label)
        return torch.stack(mask_labels, dim=0)

    def mask_tokens(self, inputs: torch.Tensor, mask_labels: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        
        # input_ids
        labels = inputs.clone()
        
        probability_matrix = mask_labels
        
        # ---------------------------- 都是 huggingface transformers 源码 ------------------------#
        special_tokens_mask = [
            self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
        ]
        probability_matrix.masked_fill_(torch.tensor(special_tokens_mask, dtype=torch.bool), value=0.0)

        masked_indices = probability_matrix.bool()

        labels[~masked_indices] = -100  
        
        # 80%的词 被替换成tokenizer.mask_token [MASK]
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)
        
        # 10%的词 被随机替换
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]
        # ---------------------------- 都是 huggingface transformers 源码 ------------------------#

        # 其余 10%的词 不变
        return inputs, labels

    def __call__(self, examples):

        input_ids_list, token_type_ids_list, attention_mask_list = list(zip(*examples))
        cur_max_seq_len = max(len(input_id) for input_id in input_ids_list)
        max_seq_len = min(cur_max_seq_len, self.max_seq_len)

        input_ids, token_type_ids, attention_mask = self.pad_and_truncate(input_ids_list,
                                                                          token_type_ids_list,
                                                                          attention_mask_list,
                                                                          max_seq_len)
        

        batch_mask = self.ngram_mask(input_ids_list, max_seq_len)
        input_ids, mlm_labels = self.mask_tokens(input_ids, batch_mask)
        data_dict = {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'labels': mlm_labels
        }

        return data_dict

![mask](https://img-blog.csdnimg.cn/6bd70eca797d4b5682baa95f6225bee9.png?x-oss-process=image/watermark,type_ZmFuZ3poZW5naGVpdGk,shadow_10,text_aHR0cHM6Ly9ibG9nLmNzZG4ubmV0L3dlaXhpbl80MTI4NzA2MA==,size_16,color_FFFFFF,t_70)

In [15]:
data_collator = Collator(max_seq_len=64, tokenizer=tokenizer, mlm_probability=0.15)

In [16]:
def check_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [17]:
logging_dir = os.path.join(output_dir, 'log')
model_save_dir = os.path.join(output_dir, 'best_model_ckpt')
tokenizer_and_config = os.path.join(output_dir, 'tokenizer_and_config')

check_dir(model_save_dir)
check_dir(logging_dir)
check_dir(tokenizer_and_config)

In [18]:
from transformers import TrainingArguments
# 
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    learning_rate=6e-5,
    num_train_epochs=1,
    per_device_train_batch_size=128,
    save_steps=2000,
    logging_steps=200,
    save_total_limit=5,
    fp16=False,
    prediction_loss_only=True,
    logging_dir=logging_dir,
    logging_first_step=True,
    dataloader_num_workers=4,
    disable_tqdm=False,
    seed=2021
    )

PyTorch: setting up devices


In [19]:
from transformers import Trainer
#
trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=train_dataset,
    )

trainer.train()

***** Running training *****
  Num examples = 34334
  Num Epochs = 1
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 1
  Total optimization steps = 269


Step,Training Loss
1,2.181547


KeyboardInterrupt: 